# Ring loading problem

In [ ]:
#@title Data and verification

import numpy as np
import itertools

solution = [(np.float64(0.8928570945587888), np.float64(0.05952382012678484)), (np.float64(0.44586335510090447), np.float64(0.5541366448990955)), (np.float64(0.48270810611962867), np.float64(0.4220537880782024)), (np.float64(0.3333332972264078), np.float64(0.6666667027735921)), (np.float64(0.2619047708053195), np.float64(0.23809522594859156)), (np.float64(0.32137957426389363), np.float64(0.6786204257361064)), (np.float64(0.7381442491666856), np.float64(0.14280815724670945)), (np.float64(0.49999999244019133), np.float64(0.49999999244019133)), (np.float64(0.14280815724670945), np.float64(0.7381442491666856)), (np.float64(0.6786204257361064), np.float64(0.32137957426389363)), (np.float64(0.23809522594859156), np.float64(0.2619047708053195)), (np.float64(0.6666667027735921), np.float64(0.3333332972264078)), (np.float64(0.4220537880782024), np.float64(0.48270810611962867)), (np.float64(0.5541366448990955), np.float64(0.44586335510090447)), (np.float64(0.05952382012678484), np.float64(0.8928570945587888))]


def validate_input(u: list[float], v: list[float]):
  """Validates the input vectors u and v."""
  if len(u) != len(v):
    raise ValueError('Input vectors u and v must have the same length.')
  if not all(isinstance(x, (float, int)) for x in u) or not all(
      isinstance(x, (float, int)) for x in v
  ):
    raise ValueError('Input vectors must contain only floats or integers.')
  if not all(x >= 0 for x in u) or not all(x >= 0 for x in v):
    raise ValueError('Input vectors must be non-negative.')
  # Use a small tolerance for floating point comparisons
  if not all(ui + vi <= 1.000001 for ui, vi in zip(u, v)):
    raise ValueError('Condition u_i + v_i <= 1 must be met for all i.')
  if any(np.isnan(x) for x in u) or any(np.isnan(x) for x in v):
    raise ValueError('Input vectors cannot contain NaN values.')


def compute_alpha_exact(u: list[float], v: list[float]) -> tuple[float, float]:
  """Computes the smallest possible alpha (the min) for the given input vectors as well as alpha's smooth version."""
  validate_input(u, v)
  m = len(u)
  if m == 0:
    return 0.0, 0.0

  best_alpha = float('inf')

  z_choices = [(vi, -ui) for ui, vi in zip(u, v)]
  for z in itertools.product(*z_choices):
    max_alpha_for_z = 0
    if m > 1:
      max_alpha_for_z = max(abs(sum(z[:k]) - sum(z[k:])) for k in range(1, m))
    best_alpha = min(best_alpha, max_alpha_for_z)

  return best_alpha


def get_score(params: list[tuple[float, float]], m: int) -> float:
  """Calculates the score for a given set of (u, v) vector parameters."""
  assert isinstance(params, list), "params must be a list"
  assert isinstance(m, int), "m must be an integer"
  assert len(params) == m, "Invalid number of parameters"

  u, v = [], []
  for item in params:
    if not isinstance(item, (tuple, list)) or len(item) != 2:
      return -1.0
    if not isinstance(item[0], (float, int)) or not isinstance(
        item[1], (float, int)
    ):
      return -1.0
    if np.isnan(item[0]) or np.isnan(item[1]):
      return -1.0

    ui, vi = abs(item[0]), abs(item[1])
    s = ui + vi
    if s > 1.0:
      ui /= s
      vi /= s
    assert s < 1e9, "Numerical instability"
    u.append(ui)
    v.append(vi)

  try:
    exact_alpha = compute_alpha_exact(u, v)
    return exact_alpha
  except (ValueError, OverflowError):
    return -1.0

get_score(solution, 15)

**Prompt used**

Act as an expert in optimization and algorithm design. Your task is to solve the following problem:

Given an integer m, find two lists of non-negative floats, u = [u_1, ..., u_m] and v = [v_1, ..., v_m], that satisfy the condition u_i + v_i <= 1 for all i. The goal is to find the lists (u, v) that maximize a value alpha.

This alpha is defined as the smallest value for which there exists a sequence z = [z_1, ..., z_m], where each z_i is chosen to be either v_i or -u_i, such that the following inequality holds for all k from 1 to m-1:

|sum(z_i for i=1 to k) - sum(z_i for i=k+1 to m)| <= alpha

You need to write a search function that takes an integer m and searches for the best lists u and v of that length. The construction you should return is a list of tuples [(u_1, v_1), (u_2, v_2), ..., (u_m, v_m)].

Your solution will be evaluated by the following scoring function, which you have access to and can call as many times as you want. The function calculates a smooth version of alpha, which you should aim to maximize.

```
def get_score(params: list[tuple[float, float]], m: int) -> float:
    """Calculates the score for a given set of (u, v) vector parameters."""
    if not isinstance(params, list) or len(params) != m:
        return -1.0

u, v = [], []
for item in params:
    ui, vi = abs(item[0]), abs(item[1])
    s = ui + vi
    if s > 1.0:
        ui /= s
        vi /= s
    u.append(ui)
    v.append(vi)

try:
    # This external function computes the score (smooth_alpha).
    exact_alpha = compute_alpha_exact(u, v)
    return exact_alpha
except (ValueError, OverflowError):
    return -1.0
```

You do not need to implement the get_score or the compute_alpha_exact functions, they are already present in the codebase, you have access to them. Your task is to implement the search function search_for_best_vectors(m). It has 2000 seconds to run. If it hasn't returned after that time, it will be terminated.

You can access the previously found best constructions through the best_vectors_10 global variable (where the 10 should be replaced by your m).

Big important hint: if your score is between 0.999 and 1.000, or between 1.099 and 1.100, it might be that you are stuck in a big local optima! Scores higher than 1.101 are easily possible, so try your best to get out of that local optimum. It is believed that the optimal score that can be achieved is 1.1111 (which is 10/9), try to aim for that if you can!

Good luck!

In [ ]:
#@title Initial program used (evolved by an earlier AlphaEvolve experiment on the same problem)

def search_for_best_vectors(m: int) -> list[tuple[float, float]]:
  """Searches for the best (u, v) vectors of length m that maximize alpha.

  A simple random search is implemented as a starting point.
  """
  variable_name = f'best_vectors_{m}'
  if variable_name in globals():
    params = globals()[variable_name]
  else:
    # Initialize randomly, tending to make u_i + v_i = 1 as per get_score implicit normalization
    params = []
    for _ in range(m):
      ui = np.random.rand()
      # Make u_i + v_i = 1 with high probability (e.g., 80%), otherwise allow < 1
      vi = (
          1.0 - ui
          if np.random.rand() < 0.8
          else np.random.uniform(0.0, 1.0 - ui)
      )
      params.append((ui, vi))

  best_score = get_score(params, m)
  best_params = copy.deepcopy(params)
  print(f'Initial score for m={m}: {best_score}')

  start_time = time.time()

  # Define the objective function for the optimizer.
  # It takes a flat numpy array and returns the negative score.
  def objective(x, m):
    params_list = [(x[i], x[m + i]) for i in range(m)]
    score = get_score(params_list, m)
    return (
        -score if score > 0 else float('inf')
    )  # Return a very large value for invalid scores.

  x_best = np.array([p[0] for p in best_params] + [p[1] for p in best_params])

  # Iterated Local Search (ILS) loop.
  total_runtime = np.random.randint(100, 2000)
  while time.time() - start_time < total_runtime:
    # 1. Perturbation: Hybrid Strategy
    x_start = x_best.copy()

    perturbation_type = np.random.rand()

    # 60% chance: Small Gaussian noise to a few elements (fine-tuning)
    if perturbation_type < 0.6:
      num_to_mutate = np.random.randint(
          1, max(2, m // 4 + 1)
      )  # Mutate 1 to m/4 elements
      indices_to_mutate = np.random.choice(m, num_to_mutate, replace=False)
      noise_std = 0.05  # Small noise
      for idx in indices_to_mutate:
        ui_current = x_start[idx]
        vi_current = x_start[idx + m]

        new_ui = np.clip(ui_current + np.random.normal(0, noise_std), 0.0, 1.0)
        # Adjust v_i to maintain sum property then re-clip, prioritizing u_i+v_i <= 1
        current_sum = ui_current + vi_current
        if current_sum > 0:  # Avoid division by zero
          # Maintain original u_i:v_i ratio for change, but within u_i+v_i<=1
          new_vi_raw = vi_current / current_sum * (new_ui + vi_current)
          new_vi = np.clip(new_vi_raw, 0.0, 1.0 - new_ui)
        else:  # If sum was 0, just make new_vi 1-new_ui
          new_vi = 1.0 - new_ui

        x_start[idx] = new_ui
        x_start[idx + m] = new_vi

    # 20% chance: Snap some elements to privileged values or new random values (stronger local jump)
    elif perturbation_type < 0.8:
      privileged_u_values = [0.0, 0.25, 0.5, 0.75, 1.0]
      num_to_mutate = np.random.randint(
          1, max(2, m // 3 + 1)
      )  # Mutate 1 to m/3 elements
      indices_to_mutate = np.random.choice(m, num_to_mutate, replace=False)
      for idx in indices_to_mutate:
        if np.random.rand() < 0.7:  # 70% chance to snap to privileged
          ui = random.choice(privileged_u_values)
        else:  # 30% chance to assign completely random
          ui = np.random.rand()

        # Tend to make u_i + v_i = 1 for these jumps, as per get_score behavior
        vi = 1.0 - ui

        x_start[idx] = ui
        x_start[idx + m] = vi

    # 10% chance: Block mutation (introduce symmetry/patterns)
    elif perturbation_type < 0.9:
      if m > 1:
        block_size = np.random.randint(
            1, min(m, 3) + 1
        )  # Mutate 1 to 3 elements
        start_idx = np.random.randint(0, m - block_size + 1)

        # Choose a common value for the block
        common_ui = np.random.rand()
        common_vi = 1.0 - common_ui  # Again, force u_i + v_i = 1 for block

        for i in range(start_idx, start_idx + block_size):
          # Add tiny noise to break exact equality sometimes, while keeping general pattern
          x_start[i] = np.clip(common_ui + np.random.normal(0, 0.01), 0.0, 1.0)
          x_start[i + m] = np.clip(
              common_vi + np.random.normal(0, 0.01), 0.0, 1.0 - x_start[i]
          )
      else:  # m=1, act like single element snap
        ui = np.random.rand()
        vi = 1.0 - ui
        x_start[0] = ui
        x_start[m] = vi

    # 10% chance: Global reset with u_i + v_i = 1 (strongest perturbation)
    else:
      for idx in range(m):
        ui = np.random.rand()
        vi = 1.0 - ui  # Force u_i + v_i = 1
        x_start[idx] = ui
        x_start[idx + m] = vi

    # 2. Local Search:
    bounds = [(0.0, 1.0)] * (2 * m)
    constraints = [
        {'type': 'ineq', 'fun': lambda x, i=i: 1.0 - x[i] - x[m + i]}
        for i in range(m)
    ]

    res = optimize.minimize(
        objective,
        x_start,
        args=(m,),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 100, 'ftol': 1e-7},
    )

    # 3. Selection:
    current_score = -res.fun
    if current_score > best_score:
      best_score = current_score
      x_best = res.x
      print(f'New best score: {best_score}')

  best_params = [(x_best[i], x_best[m + i]) for i in range(m)]
  print(f'Final score after ILS: {best_score}')
  return best_params

In [ ]:
#@title Code evolved by AlphaEvolve

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import scipy.linalg as la
import collections
import copy
import math
import numba

njit = numba.njit


def search_for_best_vectors(m: int) -> list[tuple[float, float]]:
  """Searches for the best (u, v) vectors of length m that maximize alpha.

  A simple random search is implemented as a starting point.
  """
  from scipy.fft import dct, idct
  # Initialize with a structured, symmetric guess (a linear ramp).
  # This construction satisfies u_i + v_i = 1, with v_i = u_{m-1-i}.
  if m == 1:
      u_init_ramp = np.array([0.5])
  else:
      # Linear ramp from 1/3 to 2/3. With v_i = u_{m-1-i}, this satisfies u_i+v_i=1.
      u_init_ramp = np.linspace(1/3, 2/3, m)
  params_ramp = [(u_init_ramp[i], u_init_ramp[m - 1 - i]) for i in range(m)]
  score_ramp = get_score(params_ramp, m)

  # Initialize with the theoretically-motivated 3-piece structure for 10/9 target.
  u_init_theoretical = np.zeros(m)
  # Values for 10/9 target based on theory.
  # C_L is for u_i, C_R for v_i, C_M for middle element if m is odd.
  # Since v_i = u_{m-1-i} is enforced, this implies u_i = C_L and u_{m-1-i} = C_R.
  # For symmetry u_i + u_{m-1-i} <= 1, and for specific 10/9 structure values from literature.
  C_L, C_M, C_R = 1/3, 0.5, 2/3

  N_pairs = m // 2
  for i in range(N_pairs):
      u_init_theoretical[i] = C_L
      u_init_theoretical[m - 1 - i] = C_R
  if m % 2 == 1: # Middle element for odd m
      u_init_theoretical[N_pairs] = C_M

  # Ensure the symmetric condition u_i + u_{m-1-i} <= 1 is met for this initialization
  # For the chosen C_L, C_R (1/3, 2/3), their sum is 1.0. For C_M=0.5, u_middle + v_middle = 0.5+0.5=1.0.
  # So, direct assignment is fine, no explicit clipping/scaling here.
  params_theoretical = [(u_init_theoretical[i], u_init_theoretical[m - 1 - i]) for i in range(m)]
  score_theoretical = get_score(params_theoretical, m)

  # Choose the best initial guess among ramp, theoretical, and loaded previous best.
  params = params_ramp
  best_score = score_ramp

  if score_theoretical > best_score:
      params = params_theoretical
      best_score = score_theoretical

  variable_name = f'best_vectors_{m}'
  if variable_name in globals():
    initial_params_loaded = globals()[variable_name]
    u_old = np.array([p[0] for p in initial_params_loaded])
    v_old = np.array([p[1] for p in initial_params_loaded])
    u_loaded_symmetric = np.zeros(m)
    for i in range(m):
      u_loaded_symmetric[i] = (u_old[i] + v_old[m - 1 - i]) / 2.0
    params_loaded = [(u_loaded_symmetric[i], u_loaded_symmetric[m - 1 - i]) for i in range(m)]
    score_loaded = get_score(params_loaded, m)
    if score_loaded > best_score:
      params = params_loaded
      best_score = score_loaded

  best_params = copy.deepcopy(params)
  print(f'Initial score for m={m}: {best_score}')

  start_time = time.time()
  total_runtime_budget_seconds = 1950.0 # Total runtime budget for the search part (excluding initial setup)

  # NEW PHASE: Simple Breakthrough Random Search
  # Goal: Quickly find *any* solution with score > 1.0 to escape initial low scores.
  # This phase uses a small fraction of the total budget or stops early if score > 1.0.
  simple_breakthrough_timeout_factor = 0.03 # Allocate 3% of total time for this phase (e.g., ~58.5s)
  simple_breakthrough_timeout = simple_breakthrough_timeout_factor * total_runtime_budget_seconds
  simple_breakthrough_start_time = time.time()
  num_simple_attempts = 0

  print("Starting simple breakthrough random search phase...")
  # Loop condition: within time budget AND score is still below target (1.0001 to account for float precision)
  while (time.time() - simple_breakthrough_start_time < simple_breakthrough_timeout) and best_score < 1.0001:
      num_simple_attempts += 1
      current_params = []
      for _ in range(m):
          ui = np.random.rand()
          vi = np.random.rand() # Let get_score handle the u_i + v_i > 1 normalization
          current_params.append((ui, vi))

      current_score = get_score(current_params, m)

      if current_score > best_score:
          best_score = current_score
          best_params = copy.deepcopy(current_params)
          print(f"Found new best score in simple random search: {best_score:.4f} (attempt {num_simple_attempts})")
          if best_score > 1.0: # Stop this phase early if score > 1.0 is achieved
              print("Score > 1.0 achieved, exiting simple random search early.")
              break # Exit simple breakthrough phase

  print(f"Finished simple breakthrough random search phase. Best score: {best_score:.4f} after {num_simple_attempts} attempts.")
  time_spent_in_simple_phase = time.time() - simple_breakthrough_start_time
  print(f"Time spent in simple random search: {time_spent_in_simple_phase:.2f} seconds.")

  # Now, start the aggressive random search phase, using the remaining budget.
  # Use a proportion of the *remaining* total budget, ensuring it's not too short.
  # The factor of 0.5 now applies to the time left after the simple phase.
  aggressive_random_search_timeout_factor = 0.5 # Allocate 50% of *remaining* time for aggressive random search
  remaining_time_after_simple = max(0.0, total_runtime_budget_seconds - time_spent_in_simple_phase)
  random_search_timeout = aggressive_random_search_timeout_factor * remaining_time_after_simple
  random_search_start_time = time.time() # Reset start time for aggressive phase
  num_random_attempts = 0 # Reset counter for aggressive phase
  # No fixed max_random_attempts_per_phase, rely on time and score for stopping.

  print("Starting aggressive random search phase...")
  # Aggressive random search stopping condition: Stop if time runs out OR a sufficiently good score is found.

  # Define probabilities for different random search strategies, now all symmetric.
  # Focusing on strategies that are known to yield good results for symmetric configurations.
  p_perturbed_theoretical = 0.4   # 40% chance for perturbed theoretical values (1/3, 1/2, 2/3)
  p_dct_random = 0.3              # 30% chance for smooth DCT-based generation
  p_random_symmetric_fixed_sum = 0.3 # 30% chance for simple random symmetric pairs with u_i+v_i=1

  C_L_ideal, C_M_ideal, C_R_ideal = 1/3, 0.5, 2/3 # Ideal values for 10/9 structure

  while (time.time() - random_search_start_time < random_search_timeout):
      num_random_attempts += 1

      roll = np.random.rand()
      u_new = np.zeros(m)

      if roll < p_perturbed_theoretical and m > 1:
          # Perturbed Theoretical Random Search (High probability due to effectiveness)
          noise_scale_theoretical = 0.05 + np.random.rand() * 0.08 # Variable noise: 0.05 to 0.13 (slightly higher max)

          n_half = m // 2
          for i in range(n_half):
              u_new[i] = np.clip(C_L_ideal + np.random.normal(0, noise_scale_theoretical), 0.0, 1.0)
              u_new[m - 1 - i] = np.clip(C_R_ideal + np.random.normal(0, noise_scale_theoretical), 0.0, 1.0)

              # Ensure u_i + u_{m-1-i} <= 1.0 after perturbation. Bias towards 1.0.
              current_sum = u_new[i] + u_new[m-1-i]
              target_sum = 1.0 - np.random.rand() * 0.05 # Aim slightly below 1.0, but mostly at 1.0
              if current_sum > target_sum:
                  scale_factor = target_sum / current_sum
                  u_new[i] *= scale_factor
                  u_new[m-1-i] *= scale_factor

          if m % 2 == 1:
              u_new[n_half] = np.clip(C_M_ideal + np.random.normal(0, noise_scale_theoretical), 0.0, 0.5)

          v_new = u_new[::-1]
          current_params = list(zip(u_new, v_new))

      elif roll < p_perturbed_theoretical + p_dct_random:
          # DCT-based Random Search to generate smooth candidates
          c_rand_scale = np.random.uniform(0.5, 2.0)
          decay_rate = np.random.uniform(m / 8.0, m / 2.0) if m > 1 else 1.0
          freq_weights = c_rand_scale * (0.5 ** (np.arange(m) / decay_rate))
          c_rand = np.random.normal(size=m) * freq_weights

          u_new = idct(c_rand, type=2, norm='ortho')

          # Project u to be feasible: map to [0,1] then enforce sum constraint
          min_u, max_u = np.min(u_new), np.max(u_new)
          if max_u - min_u > 1e-6:
              u_new = (u_new - min_u) / (max_u - min_u)
          u_new = np.clip(u_new, 0.0, 1.0)

          for i in range(m // 2 + 1):
              s = u_new[i] + u_new[m - 1 - i]
              if s > 1.0:
                  u_new[i] /= s
                  u_new[m - 1 - i] /= s

          v_new = u_new[::-1]
          current_params = list(zip(u_new, v_new))

      else:
          # Simple Symmetric Random with Fixed Sum (replacing unstructured)
          # Generate u_i such that u_i + u_{m-1-i} = 1 and v_i = u_{m-1-i}.
          # This greatly simplifies the search space and focuses on symmetric solutions
          # with full sum.
          n_half = m // 2
          for i in range(n_half):
              ui = np.random.rand() # ui is in [0,1]
              u_new[i] = ui
              u_new[m - 1 - i] = 1.0 - ui # Ensures u_i + u_{m-1-i} = 1

          if m % 2 == 1:
              u_new[n_half] = 0.5 # For middle element, u_middle + v_middle = 0.5 + 0.5 = 1.0

          v_new = u_new[::-1] # Enforces v_i = u_{m-1-i} due to the way u_new is constructed.
          current_params = list(zip(u_new, v_new))

      current_score = get_score(current_params, m)

      if current_score > best_score:
          best_score = current_score
          best_params = copy.deepcopy(current_params) # Store the potentially non-symmetric params if they are better
          print(f"Found new best score in random search: {best_score:.4f} (attempt {num_random_attempts})")
          # If a very good score is found, we can potentially exit early from aggressive random search
          # to give more time to ILS. The hint mentions 1.101+ is good.
          if best_score >= 1.101:
              print("Score >= 1.101 achieved, exiting aggressive random search early.")
              break # Exit random search early if we hit a high score

  print(f"Finished aggressive random search phase. Best score: {best_score:.4f} after {num_random_attempts} attempts.")
  print(f"Time spent in random search: {time.time() - random_search_start_time:.2f} seconds.")

  u_current_from_best_params = np.array([p[0] for p in best_params])
  v_current_from_best_params = np.array([p[1] for p in best_params])

  u_best_initial = np.zeros(m)
  for i in range(m):
      # Average u_i and v_{m-1-i} from the current best to enforce symmetry.
      # This creates a symmetric starting point for the ILS.
      u_best_initial[i] = (u_current_from_best_params[i] + v_current_from_best_params[m - 1 - i]) / 2.0
  u_best_initial = np.clip(u_best_initial, 0.0, 1.0)  # Ensure it's within bounds after averaging

  # Convert the best u-space vector to DCT-space (c-space). The ILS will operate on c.
  c_best = dct(u_best_initial, type=2, norm='ortho')

  # Initialize a buffer to store successful c vectors for future diversification
  memory_buffer = [c_best.tolist()]
  memory_size = 5

  # Define the objective function for the optimizer, operating in DCT-space.
  # It takes DCT coefficients `c`, transforms them to `u`, projects to the
  # feasible set, and then returns the negative score.
  def objective_dct(c, m):
    # Transform from DCT-space (c) to vector-space (u)
    u = idct(c, type=2, norm='ortho')

    # Project u back into the feasible set.
    u = np.clip(u, 0.0, 1.0)
    for i in range(m // 2 + 1):
        s = u[i] + u[m - 1 - i]
        if s > 1.0:
            # Scale to satisfy the constraint u_i + u_{m-1-i} <= 1
            # Note: s will not be zero because u elements are non-negative and s > 1.
            u[i] /= s
            u[m-1-i] /= s

    v = u[::-1] # Enforce symmetry v_i = u_{m-1-i}
    params_list = list(zip(u, v))
    score = get_score(params_list, m)
    return -score if score > 0 else float('inf')

  # Iterated Local Search (ILS) loop.
  # Constraints are now handled inside objective_dct by projection, so the
  # explicit constraints for the optimizer are removed.
  constraints = []

  # Parameters for adaptive noise and advanced perturbation
  unimproved_iterations_counter, max_unimproved_iterations = 0, 20 # Reduced from 30 to 20 for faster noise adaptation
  current_noise_scale, noise_scale_min, noise_scale_max = 0.03, 0.005, 0.25 # Slightly higher initial noise (0.02->0.03), higher max (0.20->0.25)
  noise_scale_factor_increase, noise_scale_factor_decrease = 1.4, 0.7 # More aggressive scaling (1.3->1.4, 0.8->0.7)

  # Probabilities for ILS perturbation strategies in DCT-space.
  p_memory = 0.10  # Jump to a solution from memory
  p_full_random_restart = 0.05 # Complete random restart
  p_crossover_dct = 0.05 # Crossover with a random or memory solution in DCT space
  # The remaining 80% of perturbations will be frequency-dependent Gaussian noise on the best solution.

  # ILS loop. Check remaining_time_for_ils.
  # The `start_time` is the overall start time of the function.
  while (time.time() - start_time) < total_runtime_budget_seconds:
    # 1. Perturbation Strategy: Perturb in DCT-space (c-space)
    c_start = c_best.copy()
    roll = np.random.rand()

    if roll < p_memory and memory_buffer: # Jump to a past good solution
        c_start = np.array(random.choice(memory_buffer))
    elif roll < p_memory + p_full_random_restart: # Full random restart in c-space
        # Create a random, feasible u-vector and transform it to c-space.
        u_rand = np.random.rand(m)
        for i in range(m // 2 + 1):
            s = u_rand[i] + u_rand[m - 1 - i]
            if s > 1.0:
                u_rand[i] /= s
                u_rand[m - 1 - i] /= s
        c_start = dct(u_rand, type=2, norm='ortho')
    elif roll < p_memory + p_full_random_restart + p_crossover_dct: # Crossover in c-space
        if memory_buffer and np.random.rand() < 0.5: # Crossover with a solution from memory
            c_partner = np.array(random.choice(memory_buffer))
        else: # Crossover with a newly generated random solution
            u_rand_partner = np.random.rand(m)
            for i in range(m // 2 + 1):
                s = u_rand_partner[i] + u_rand_partner[m - 1 - i]
                if s > 1.0:
                    u_rand_partner[i] /= s
                    u_rand_partner[m - 1 - i] /= s
            c_partner = dct(u_rand_partner, type=2, norm='ortho')

        # Perform crossover: mix coefficients from c_best and c_partner
        # Choose a random split point or blend coefficients. Blending is usually better.
        alpha_blend = np.random.uniform(0.2, 0.8) # Blend ratio
        c_start = alpha_blend * c_best + (1 - alpha_blend) * c_partner
    else:
        # The main perturbation is frequency-dependent Gaussian noise.
        # The std dev of the noise decays exponentially with frequency, making
        # large, smooth changes more likely than small, noisy ones.
        decay_rate = m / 4.0 if m > 0 else 1.0
        freq_weights = 0.5 ** (np.arange(m) / decay_rate)
        noise_std_devs = current_noise_scale * freq_weights
        c_start += np.random.normal(0, noise_std_devs)

        # Add a small chance of swapping two coefficients to explore different structures.
        # Add a small chance of swapping two coefficients to explore different structures.
        if m > 1 and np.random.rand() < 0.05: # Reduced probability
            idx1, idx2 = np.random.choice(m, 2, replace=False)
            c_start[idx1], c_start[idx2] = c_start[idx2], c_start[idx1]

        # Add a small chance of sparsifying higher frequency DCT coefficients.
        if m > 2 and np.random.rand() < 0.05: # 5% chance
            num_to_zero = np.random.randint(1, max(2, m // 4)) # Zero out 1 to m/4 coefficients
            # Select higher frequency coefficients to zero out.
            # DCT coefficients are ordered by frequency, so higher indices are higher frequencies.
            high_freq_indices = np.random.choice(np.arange(m // 2, m), num_to_zero, replace=False)
            c_start[high_freq_indices] = 0.0

    # 2. Local Search (only if enough time remains)
    if (time.time() - start_time) >= total_runtime_budget_seconds:
        break # Exit loop if time runs out before starting local search

    # The optimizer works on `c`, which is unconstrained.
    bounds = [(None, None)] * m
    res = optimize.minimize(objective_dct, c_start, args=(m,), method='SLSQP', bounds=bounds,
                            constraints=constraints, options={'maxiter': 500, 'ftol': 1e-10, 'disp': False})

    # 3. Selection
    current_score = -res.fun
    if res.success and current_score > best_score:
      best_score, c_best = current_score, res.x
      memory_buffer.append(c_best.tolist())
      if len(memory_buffer) > memory_size:
          memory_buffer.pop(0)
      print(f'New best score: {best_score:.6f} (Noise: {current_noise_scale:.3f})')
      unimproved_iterations_counter = 0
      current_noise_scale = max(noise_scale_min, current_noise_scale * noise_scale_factor_decrease)
    else:
        unimproved_iterations_counter += 1
        if unimproved_iterations_counter >= max_unimproved_iterations:
            current_noise_scale = min(noise_scale_max, current_noise_scale * noise_scale_factor_increase)
            unimproved_iterations_counter = 0

  # Convert the final best c-vector back to a u-vector.
  u_best = idct(c_best, type=2, norm='ortho')
  # Project one last time to ensure feasibility due to potential floating point inaccuracies.
  u_best = np.clip(u_best, 0.0, 1.0)
  for i in range(m // 2 + 1):
      s = u_best[i] + u_best[m-1-i]
      if s > 1.0:
          u_best[i] /= s
          u_best[m-1-i] /= s

  # Construct the final parameters from the best u vector found.
  v_best = u_best[::-1]
  best_params = list(zip(u_best, v_best))
  print(f'Final score after ILS: {best_score:.4f}')
  return best_params


In [ ]:
#@title Verification code



def validate_input(u: list[float], v: list[float]):
  """Validates the input vectors u and v."""
  if len(u) != len(v):
    raise ValueError('Input vectors u and v must have the same length.')
  if not all(isinstance(x, (float, int)) for x in u) or not all(
      isinstance(x, (float, int)) for x in v
  ):
    raise ValueError('Input vectors must contain only floats or integers.')
  if not all(x >= 0 for x in u) or not all(x >= 0 for x in v):
    raise ValueError('Input vectors must be non-negative.')
  # Use a small tolerance for floating point comparisons
  if not all(ui + vi <= 1.000001 for ui, vi in zip(u, v)):
    raise ValueError('Condition u_i + v_i <= 1 must be met for all i.')
  if any(np.isnan(x) for x in u) or any(np.isnan(x) for x in v):
    raise ValueError('Input vectors cannot contain NaN values.')


def compute_alpha_exact(u: list[float], v: list[float]) -> tuple[float, float]:
  """Computes the smallest possible alpha (the min) for the given input vectors as well as alpha's smooth version."""
  validate_input(u, v)
  m = len(u)
  if m == 0:
    return 0.0, 0.0

  best_alpha = float('inf')

  z_choices = [(vi, -ui) for ui, vi in zip(u, v)]
  for z in itertools.product(*z_choices):
    max_alpha_for_z = 0
    if m > 1:
      max_alpha_for_z = max(abs(sum(z[:k]) - sum(z[k:])) for k in range(1, m))
    best_alpha = min(best_alpha, max_alpha_for_z)

  return best_alpha


def get_score(params: list[tuple[float, float]], m: int) -> float:
  """Calculates the score for a given set of (u, v) vector parameters."""
  if not isinstance(params, list) or len(params) != m:
    return -1.0

  # Cap m to avoid excessive computation time from the O(2^m) algorithm.
  if len(params) != m:
    return -1.0
  if m > 18:
    return -1.0

  u, v = [], []
  for item in params:
    if not isinstance(item, (tuple, list)) or len(item) != 2:
      return -1.0
    if not isinstance(item[0], (float, int)) or not isinstance(
        item[1], (float, int)
    ):
      return -1.0
    if np.isnan(item[0]) or np.isnan(item[1]):
      return -1.0

    ui, vi = abs(item[0]), abs(item[1])
    s = ui + vi
    if s > 1.0:
      ui /= s
      vi /= s
    if s > 1e9:
      return -1.0
    u.append(ui)
    v.append(vi)

  # negative score if variance of the u-v is small
  diff_variance = np.var(np.array(u) - np.array(v))
  if diff_variance < 0.1:
    return -1.0

  # I noticed that local optima have a small value somewhere
  if min(u) < 0.05 or min(v) < 0.05:
    return -1.0

  # if all u[i]+v[i] sums are > 0.99, that's a boring local optima
  if all(ui + vi > 0.95 for ui, vi in zip(u, v)):
    return -1.0

  # I also noticed that if the first two or last two entries are equal, that's a boring local optima, so let's push away from that
  if (
      np.abs(u[0] - u[1]) < 0.05
      or np.abs(v[0] - v[1]) < 0.05
      or np.abs(u[-2] - u[-1]) < 0.05
      or np.abs(v[-2] - v[-1]) < 0.05
  ):
    return -1.0

  try:
    exact_alpha = compute_alpha_exact(u, v)
    return exact_alpha
  except (ValueError, OverflowError):
    return -1.0
